<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/master/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Enunciado](https://www.u-cursos.cl/ingenieria/2020/1/CC3001/1/tareas/r/1_CC3001-Tarea2-Diagrama_de_estados.pdf)

#Introducción
Este notebook tiene el objetivo de ilustrar el uso de diagramas de estado en la programación de una calculador simple tipo Maple o Matlab. Las condiciones son restrictivas, lo que facilita la implementación del mismo. Hay dos comandos, o bien asignar en la memoria una variable (incluyendo operaciones matemáticas) o bien imprimir una variable ya registrada en pantalla. Las operaciones permitidas son $+$, $-$, $*$, $/$ y ^. Sólo se operarán números enteros (la división por lo mismo será entera). Las variables pueden partir con una letra e ir seguidas por letras, números o '_'. Además, no habrá paréntesis, por lo que se evalurán las expresiones de izquierda a derecha (a partir del $=$).

#Diagramas de Estado

El diagrama de estado más simple es el que se dibuja a nivel macro. Existen dos estado que ve el usuario: In (que es cuando se le pide ingresar un comando) y Out (el resultado del comando que ha sido escrito). En el enunciado no se pide que este ciclo tenga término, por lo que se puede realizar un bucle infinito usando 

```
while True:
  In
  Out
```



![Entrada y Salida](https://github.com/DiegoRomanCortes/AED/blob/master/Entrada-Salida.png?raw=true)

Todo el aparataje está contenido en el estado In, pues Out sólo imprime en pantalla lo procesado por In. Así, el lector comienza a leer el String proporcionado por el usuario. El primer caracter puede ser el inicio de una variable, el inicio de un numero, un espacio u otro caracter (error de sintaxis no expreso en el diagrama). Como el comando tipo es del tipo 
```
variable = expresion
``` 
y si se omite el '=' se entiende que se quiere imprimir en pantalla la variable, se necesita que el primer Estado no Espacio al que ingrese el lector sea Variable, de lo contrario será un error de sintaxis (no expreso en el diagrama).

Acto seguido, el lector debe asegurarse que el siguiente Estado no Espacio sea una Operacion (en concreto, el caracter debe ser '='). Tanto si el comando era de asignación como si era de impresión, cualquier otro Estado debe proporionar un error de sintaxis.

Se almacena la variable por definir como
```
variablepd
```
si el comando es de asignación. Si el comando es de impresión en pantalla de una variable ya definida, se lee como si fuera una variable más (explicado con mayor detalle posteriormente). Otros casos están contemplados en el error de sintaxis.

A nivel general, el programa funciona almacenando la parte derecha de la igualdad en una variable llamada


```
expresion
```
la cual es asignada al haber llegado a un Estado Espacio o a un Estado Operacion (operación incluye las operaciones anteriormente detalladas y '=') y al tener en memoria otra variable llamada 

```
aux
```
que simboliza el lado derecho de una operación.

El lector termina cuando detecta el último Estado Espacio (el programa se asegura de esto concatenando un espacio al String original) y hace la asignación descrita en el párrafo anterior.

Finalmente, se añade al diccionario 

```
dic
```
la variable por definir de la forma:

```
dic[variablepd] = expresion
```
Si el comando era de impresión, simplemente se ignora el cálculo de 

```
expresion
```
y se imprime el valor que ya tenía asignada la variable (que si está mal escrita arroja un error de sintaxis incorporado en el lector).



![Lector](https://github.com/DiegoRomanCortes/AED/blob/master/Lector%20(2).png?raw=true)

El comportamiento del programa cuando entra al Estado Variable es el siguiente:
![Estado Variable](https://github.com/DiegoRomanCortes/AED/blob/master/Variable.png?raw=true)

Como se aprecia en el diagrama, se parte desde Inicio (se llega hasta aquí si el caracter es una letra). No se sale del Estado Variable mientras el caracter no deje de ser un caracter permitido para la escritura de variables (una letra, un número o un guión bajo). Cualquier caracter distinto arrojará un error de sintaxis. Se sale del Estado Variable si el caracter es un espacio (entrado en el Estado Espacio) o si es una operacion (entrando en el Estado Operacion cuya flecha no aparece en el diagrama). Si 

```
variable
```
está en memoria, se reemplaza su valor y se añade a 

```
expresion
```
o a

```
aux
```


dependiendo de 

```
operacion
```
Si no está en memoria, existen dos casos: O es una variable por definir (es decir, se está en el lado izquierdo de la igualdad), o es un error de variable indefinda (si se está en el lado derecho).

Para el estado Numero, el proceso es análogo a excepción de que no se arroja el error de variable indefinida ni se guarda en memoria una variable. Esto es, al salir del Estado Número, se reemplaza su valor y se añade dependiendo de la operación.

Cuando hay Error de Sintaxis se activa un booleano que inicia la lectura del error desde donde fue activado.

El Error de Variable Indefinida detiene de inmediato la ejecución del ciclo e imprime la variable que no se encuentra en memoria.



In [26]:
dic = {}

Enteros = ['0', '1', '2', '3', '4', '5', '6', '7', '8' ,'9']

Letras = ['a','b','c','d','e','f','g',\
          'h','i','j','k','l','m','n',\
          'o','p','q','r','s','t','u',\
          'v','w','x','y','z',\
          'A', 'B', 'C', 'D', 'E', 'F',\
          'G', 'H', 'I', 'J', 'K', 'L', \
          'M', 'N', 'O', 'P', 'Q', 'R', \
          'S', 'T', 'U', 'V', 'W', 'X', \
          'Y', 'Z']

Operaciones = ['+','-','*','/','^','=']

Caracteres = Enteros + Letras + ['_']

def pedirEntrada():
  Entrada = input()
  return Entrada

def leerEntrada(Entrada):
  Entrada += ' '
  variablepd = '' #variable por definir
  variable = ''
  numero = ''
  operacion = ''
  expresion = '' #todo hasta la izquierda de un operador
  aux = '' #guarda el valor a la derecha de una operacion

  hayError = False
  hayIgual = False
  error = ''

  for char in Entrada:
    if char in Letras and variable == '' and numero == '': #Es el comienzo de una variable
      variable += char
    elif char in Caracteres and variable != '': #sigue recorriendo una variable
      variable += char
    elif (char in Operaciones or char == ' ') and (variable != '' or numero != '' or expresion != ''): #termino de recorrer una variable o numero:     
      if char == '=' and operacion == '' and variablepd == '': #comando de asignacion
        variablepd = variable
        variable = '' #ahora se puede leer otra eventual variable
        operacion = '=' #si sale de nuevo un = será error definido mas adelante
        hayIgual = True
      elif char != '=' and variable in dic and variablepd != '': #operar una variable ya definida
        aux = dic[variable]
        variable = ''
      elif numero != '': #acaba de terminar de leer un numero
        aux = int(numero)
        numero = ''
      elif char == '=' and variable in dic:
        hayError = True
      elif char != '=' and variable not in dic and variable != '' and variablepd != '':
        print('ERROR: Variable indefinida "', variable, '"', sep = '')
        break #Error de variable indefinida
      elif char == '=' and operacion != '':
        hayError = True
      if operacion != '' and (aux != '') and hayIgual: 
        if operacion == '=':
          expresion = aux
          aux = ''
        elif operacion == '+':
          expresion = expresion + aux
        elif operacion == '-':
          expresion = expresion - aux
        elif operacion  == '*':
          expresion = expresion * aux
        elif operacion == '/':
          expresion = expresion//aux
        elif operacion == '^':
          expresion = expresion**aux
        operacion = ''
        aux = ''
      if char in Operaciones:
        if not hayIgual:
          hayError=True
        operacion = char #es necesario guardar las operaciones anteriores para despues operaralas
    elif char in Enteros and variable == '': #recorriendo un numero, no una variable
      numero += char
    elif char == ' ':
      pass
    else:
      hayError = True
    if hayError:
      error += char
  if hayError:
    print('ERROR: al procesar "', error, '"', sep='')
    return
  else:
    if not hayIgual: #entonces no hubo = en todo el comando y se interpreta como printeo
      if variable not in dic:
          print('ERROR: variable indefinida "', variable, '"', sep = '')
      else:
          expresion = dic[variable]
    else:
      dic[variablepd] = expresion
  print(expresion)

leerEntrada('a = 4')
leerEntrada('a')

4
4
